In [3]:
# 构建Unet模型
import tensorflow as tf 

def unet(lonpts,latpts,channels=1): 
    inputs = tf.keras.layers.Input((lonpts,latpts,channels),name='inputs')
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same',name='conv1_1')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same',name='conv1_2')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),name='pool1')(conv1)

    conv2 = tf.keras.layers.Conv2D(64*2, (3, 3), activation='relu', padding='same',name='conv2_1')(pool1)                                                                    
    conv2 = tf.keras.layers.Conv2D(64*2, (3, 3), activation='relu', padding='same',name='conv2_2')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = tf.keras.layers.Conv2D(128*2, (3, 3), activation='relu', padding='same',name='conv3_1')(pool2)                                                                    
    conv3 = tf.keras.layers.Conv2D(128*2, (3, 3), activation='relu', padding='same',name='conv3_2')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = tf.keras.layers.Conv2D(256*2, (3, 3), activation='relu', padding='same',name='conv4_1')(pool3)                                                                    
    conv4 = tf.keras.layers.Conv2D(256*2, (3, 3), activation='relu', padding='same',name='conv4_2')(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),name='pool4')(conv4)

    conv5 = tf.keras.layers.Conv2D(512*2, (3, 3), activation='relu', padding='same',name='conv5_1')(pool4)                                                                    
    conv5 = tf.keras.layers.Conv2D(512*2, (3, 3), activation='relu', padding='same',name='conv5_2')(conv5)

    up5 = tf.keras.layers.UpSampling2D(size=(2, 2),name='up5')(conv5)
    up5_conv = tf.keras.layers.Conv2D(256*2,2,padding='same',name='up5_conv')(up5)
    conv4_feature = tf.image.resize(conv4,  
                                    (tf.shape(up5_conv)[1],tf.shape(up5_conv)[2]), 
                                     tf.image.ResizeMethod.NEAREST_NEIGHBOR,name='conv4_feature')
    concat1 = tf.keras.layers.Concatenate(axis=-1,name='concat1')([up5_conv,conv4_feature])

    conv6_1  = tf.keras.layers.Conv2D(256*2, (3, 3), activation='relu', padding='same',name='conv6_1')(concat1)                                                                    
    conv6_2  = tf.keras.layers.Conv2D(256*2, (3, 3), activation='relu', padding='same',name='conv6_2')(conv6_1)

    up6 = tf.keras.layers.UpSampling2D(size=(2, 2),name='up6')(conv6_2)
    up6_conv = tf.keras.layers.Conv2D(128*2,2,padding='same',name='up6_conv')(up6)
    conv3_feature = tf.image.resize(conv3, 
                                    (tf.shape(up6_conv)[1],tf.shape(up6_conv)[2]), 
                                     tf.image.ResizeMethod.NEAREST_NEIGHBOR,name='conv3_feature')
    concat2 = tf.keras.layers.Concatenate(axis=-1,name='concat2')([up6_conv,conv3_feature])
    
    conv7_1  = tf.keras.layers.Conv2D(128*2, (3, 3), activation='relu', padding='same',name='conv7_1')(concat2) 
    conv7_2 = tf.keras.layers.Conv2D(128*2, (3, 3), activation='relu', padding='same',name='conv7_2')(conv7_1)

    up7 = tf.keras.layers.UpSampling2D(size=(2, 2),name='up7')(conv7_2)
    up7_conv = tf.keras.layers.Conv2D(64*2,2,padding='same',name='up7_conv')(up7)
    conv2_feature = tf.image.resize(conv2,                                   
                                    (tf.shape(up7_conv)[1],tf.shape(up7_conv)[2]), 
                                     tf.image.ResizeMethod.NEAREST_NEIGHBOR,
                                     name='conv2_feature')
    concat3 = tf.keras.layers.Concatenate(axis=-1,name='concat3')([up7_conv,conv2_feature])
    
    conv8_1  = tf.keras.layers.Conv2D(64*2, (3, 3), activation='relu', padding='same',name='conv8_1')(concat3)
    conv8_2 = tf.keras.layers.Conv2D(64*2, (3, 3), activation='relu', padding='same',name='conv8_2')(conv8_1)


    up8 = tf.keras.layers.UpSampling2D(size=(2, 2),name='up8')(conv8_2)
    up8_conv = tf.keras.layers.Conv2D(32*2,2,padding='same',name='up8_conv')(up8)
    conv1_feature = tf.image.resize(conv1,                                   
                                    (tf.shape(up8_conv)[1],tf.shape(up8_conv)[2]), 
                                     tf.image.ResizeMethod.NEAREST_NEIGHBOR,
                                     name='conv1_feature')
    concat4 = tf.keras.layers.Concatenate(axis=-1,name='concat4')([up8_conv,conv1_feature])
    
    conv9_1  = tf.keras.layers.Conv2D(32*2, (3, 3), activation='relu', padding='same',name='conv9_1')(concat4)
    conv9_2 = tf.keras.layers.Conv2D(32*2, (3, 3), activation='relu', padding='same',name='conv9_2')(conv9_1)

    output = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid',name='output')(conv9_2)
    model = tf.keras.Model(inputs=inputs, outputs=output,) 
    return model



In [4]:
model = unet(128,128)
# model.summary()


In [34]:
# 准备数据集
import numpy as np
np.random.seed(0)
x_train =  np.arange(50,100, 50/(60*128*128)).reshape( 60,128, 128)   # 训练集输入数据
y_train = x_train * 2 + 1 
x_test = np.arange(0,50, 50/(60*128*128)).reshape( 60,128, 128)  # 测试集输入数据

y_test = x_test * 2 + 1   # 测试集标签数据
# print(x_train)


In [38]:
model = unet(128,128)
# model.summary()             # 打印模型结构

# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

# 训练模型
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# 测试模型
loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)

# 保存模型
model.save('self_Unet.h5')

# # 加载模型
# from tensorflow.keras.models import load_model
# model = load_model('self_Unet.h5')      

# # 继续训练模型
# model.fit(x_train, y_train, batch_size=32,epochs=10, validation_data=(x_test, y_test))






Epoch 1/10
2/2 [==============================] - 44s 25s/step - loss: 23490.4648 - mean_squared_error: 23490.4648 - val_loss: 3333.3284 - val_mean_squared_error: 3333.3284
Epoch 2/10
2/2 [==============================] - 39s 24s/step - loss: 23333.3184 - mean_squared_error: 23333.3184 - val_loss: 3333.3284 - val_mean_squared_error: 3333.3284
Epoch 3/10
2/2 [==============================] - 43s 27s/step - loss: 23333.3164 - mean_squared_error: 23333.3164 - val_loss: 3333.3284 - val_mean_squared_error: 3333.3284
Epoch 4/10
2/2 [==============================] - 50s 32s/step - loss: 23333.3164 - mean_squared_error: 23333.3164 - val_loss: 3333.3284 - val_mean_squared_error: 3333.3284
Epoch 5/10
2/2 [==============================] - 49s 29s/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/10
2/2 [==============================] - 52s 31s/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/10
2/2